In [32]:
#Skripte laden
%reload_ext autoreload
%autoreload 2
import os
os.chdir('/home/safiul/email-classification')
import data
from Email_Classification.Functions.packages import *
import Email_Classification.Functions.Functions as fn
import Email_Classification.Functions.Helper as hp
import Email_Classification.Functions.parameters as par

In [34]:
#Parameterwerte
param_process = par.params['param_process']
param_nlp = par.params['param_nlp']
param_embed = par.params['param_embed']
param_conv = par.params['param_conv']
param_lstm = par.params['param_lstm']
param_optimizer = par.params['param_optimizer']
param_fit = par.params['param_fit']
prior_params = par.params['prior_params']

In [ ]:
#Parameterwerte der besten C-LSTM-Modelle
#with open('Email_Classification/Models/Sequential Models/rnn/lstm/Models/filters/2020-10-21_1035_filter_200_unit_200_acc_0.7045/param.pkl', 'rb') as f:
#    params = pickle.load(f)
# param_process = params['param_process']
# param_nlp = params['param_nlp']
# param_embed = params['param_embed']
# param_conv = params['param_conv']
# param_lstm = params['param_lstm']
# param_optimizer = params['param_optimizer']
# param_fit = params['param_fit']
# prior_params = params['prior_params']

In [57]:
import shutil 
shutil.make_archive('Email_Classification/final_scripts','zip','Email_Classification/final_scripts')

In [ ]:
%%time
#Dateien laden
headers = pd.read_parquet(data.HEADERS_FILE)
bodies = pd.read_parquet(data.BODIES_FILE)
targets = features.zedsets.prepare_target_most_common(
    pd.read_parquet(data.TARGETS_FILE),
    target_column='target_category',
    num_most_common_class=100
)
#Der E-Mail-Datensatz mit den notwendigen Spalten erstellen
emails = fn.createDataFrameEmail(headers = headers, 
                                 bodies = bodies, 
                                 targets = targets, 
                                 sample_size = param_process['sample_size'], 
                                 seed = 125)
#del headers, bodies, targets

In [ ]:
%%time
#Der Datensatz auf Test- und Trainingsmenge aufteilen
X_train, X_test, y_train, y_test = \
model_selection.train_test_split(emails.emails, 
                                 emails.target_category, 
                                 test_size =  param_process['test_size'], 
                                 shuffle = True,
                                 random_state = 123)
#Konvertieren von kategorialen Labels in Zahlen
LE = preprocessing.LabelEncoder()
LE.fit(emails.target_category)
y_train = LE.transform(y_train)
y_test = LE.transform(y_test)
encoder = LabelBinarizer()
encoder_fit = encoder.fit(y_train)
y_train = encoder_fit.transform(y_train)
y_test = encoder_fit.transform(y_test)
del emails

In [ ]:
%%time
#Datenvorverarbeiten
TP = fn.Text_process(nchar = param_process['nchar'], 
                     rmDigits = param_process['rmDigits'], 
                     trans_lower= param_process['trans_lower'])
TP.fit(X_train)
X_train_TP = fn.multiprocess_array(TP, X_train)
X_test_TP = fn.multiprocess_array(TP, X_test)
X_train_TP = pd.concat(X_train_TP)
X_test_TP = pd.concat(X_test_TP)

X_train_TP_ = X_train_TP[X_train_TP.apply(lambda x: len(x.split())) != 0]
X_test_TP_ = X_test_TP[X_test_TP.apply(lambda x: len(x.split())) != 0]
y_train = y_train[X_train_TP.apply(lambda x: len(x.split())) != 0]
y_test = y_test[X_test_TP.apply(lambda x: len(x.split())) != 0]
del X_train, X_test, X_train_TP, X_test_TP

In [ ]:
%%time
#tokenize + padding 
Embeddings = fn.Embedding_custom(max_feature = param_embed['input_dim'],
                           X_train= X_train_TP_,
                           X_test= X_test_TP_,
                           quantile = param_embed['quantile'])
MAXLEN, tokenizer, X_train_vec, X_test_vec =  Embeddings.padding()
#Embedding-Matrix erstellen
fb_model = load_facebook_model("fasttext/cc.de.300.bin")
embedding_matrix = Embeddings.embedding_matrix(fb_model)
#with open('Email_Classification/Models/Sequential Models/rnn/lstm/Models/\
#2020-10-05_1932_acc_0.6889/feature/embedding_matrix.npy', 'rb') as f:
#    embedding_matrix = np.load(f)
del fb_model

In [ ]:
#Input des besten C-LSTM
# with open('Email_Classification/Models/Sequential Models/rnn/lstm/Models/\
#filters/2020-10-21_1035_filter_200_unit_200_acc_0.7045/feature/X_train_vec.npy', 'rb') as f:
#     X_train_vec = np.load(f)
# with open('Email_Classification/Models/Sequential Models/rnn/lstm/Models/\
#filters/2020-10-21_1035_filter_200_unit_200_acc_0.7045/feature/X_test_vec.npy', 'rb') as f:
#     X_test_vec = np.load(f)
# with open('Email_Classification/Models/Sequential Models/rnn/lstm/Models/\
#filters/2020-10-21_1035_filter_200_unit_200_acc_0.7045/feature/y_train.npy', 'rb') as f:
#     y_train = np.load(f)
# with open('Email_Classification/Models/Sequential Models/rnn/lstm/Models/filters/\
#2020-10-21_1035_filter_200_unit_200_acc_0.7045/feature/y_test.npy', 'rb') as f:
#     y_test = np.load(f)
# with open('Email_Classification/Models/Sequential Models/rnn/lstm/Models/\
#filters/2020-10-21_1035_filter_200_unit_200_acc_0.7045/feature/embedding_matrix.npy', 'rb') as f:
#     embedding_matrix = np.load(f)

In [148]:
%%time
#RNN-Modelle initialisieren
rnn_model = fn.RNN_custom(param_embed = param_embed, 
                           param_conv = param_conv, 
                           param_lstm = param_lstm, 
                           prior_params = prior_params,
                           param_optimizer = param_optimizer, 
                           param_fit = param_fit,
                           method = 'lstm', 
                           run_vi = True,
                           conv_layer = True, 
                           embedding_matrix = embedding_matrix, 
                           training = False, 
                           MAXLEN = 396,
                           run_dropout2 = False, 
                           training2 = False,
                           train_size = len(X_train_vec))
#Zusammenfassung des Modells
rnn_model.summary()

CPU times: user 323 ms, sys: 257 ms, total: 580 ms
Wall time: 278 ms


In [149]:
#Callbacks erstellen
now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
wkd = param_fit['wkd']
PATH_CHECKPOINTS = wkd + now
if not os.path.isdir(PATH_CHECKPOINTS):
    os.mkdir(PATH_CHECKPOINTS)
    
# Create callbacks
callbacks = [
    EarlyStopping(monitor= param_fit['monitor'], 
                     patience=  param_fit['patience'], 
                     min_delta= param_fit['min_delta'], 
                     mode= param_fit['mode']
),
    ModelCheckpoint(
        #see https://machinelearningmastery.com/check-point-deep-learning-models-keras/
        PATH_CHECKPOINTS +  "/weights.best.hdf5",
        monitor=  param_fit['monitor'],
        mode = param_fit['mode'],
        save_best_only= param_fit['save_best_only'],
        verbose= param_fit['verbose']
    )
]

In [ ]:
%%time
# Modell fitten
steps_per_epoch = int(np.floor((len(X_train_vec) / param_fit['batch_size'])))
print(f"Model Params.\nbatch_size: {param_fit['batch_size']}\nEpochs: {param_fit['epochs']}\n"
    f"Step p. Epoch: {steps_per_epoch}\n"
)

story = rnn_model.fit(
    X_train_vec,
    y_train,
    batch_size = param_fit['batch_size'],
    epochs= param_fit['epochs'],
    steps_per_epoch=steps_per_epoch,
    callbacks=callbacks,
    validation_data=(X_test_vec, y_test)
)

In [ ]:
#Das Mosell mit den besten Gewichte aktualisieren
rnn_model.load_weights('Email_Classification/Models/Sequential Models\
/rnn/lstm/Checkpoint/2020-12-20_1926/weights.best.hdf5')
score = rnn_model.evaluate(X_test_vec, 
                      y_test, 
                      verbose=1)
print('Test loss:    ', score[0])
print('Test accuracy:', score[1])

In [ ]:
%%time
#Embedding-Schicht mittrainieren
rnn_model.layers[1].trainable = True
rnn_model.compile(loss= 'categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
steps_per_epoch = int(np.floor((len(X_train_vec) / param_fit['batch_size'])))
print(f"Model Params.\nbatch_size: {param_fit['batch_size']}\nEpochs: {param_fit['epochs']}\n"
    f"Step p. Epoch: {steps_per_epoch}\n")
      
story = rnn_model.fit(
    X_train_vec,
    y_train,
    batch_size = param_fit['batch_size'],
    epochs = 15,
    steps_per_epoch=steps_per_epoch,
    callbacks = callbacks,
    validation_data=(X_test_vec, y_test))

In [ ]:
#Modell mit der besten Gewichte speichern
rnn_model.load_weights('Email_Classification/Models/Sequential Models\
/rnn/lstm/Checkpoint/2020-12-20_1926/weights.best.hdf5')
path = 'Email_Classification/Models/Sequential Models/rnn/lstm/Checkpoint/2020-12-20_1926'
rnn_model.save(path)

In [168]:
#Trainings- und Testmenge und Hyperparameterwerte speichern
with open('Email_Classification/Models/Sequential Models/rnn/lstm/Checkpoint/\
2020-12-20_1926/feature/X_train_vec.npy', 'wb') as f:
    np.save(f, X_train_vec)
with open('Email_Classification/Models/Sequential Models/rnn/lstm/Checkpoint/\
2020-12-20_1926/feature/X_test_vec.npy', 'wb') as f:
    np.save(f, X_test_vec)
with open('Email_Classification/Models/Sequential Models/rnn/lstm/Checkpoint/\
2020-12-20_1926/feature/y_train.npy', 'wb') as f:
    np.save(f, y_train)
with open('Email_Classification/Models/Sequential Models/rnn/lstm/Checkpoint/\
2020-12-20_1926/feature/y_test.npy', 'wb') as f:
    np.save(f, y_test)
with open('Email_Classification/Models/Sequential Models/rnn/lstm/Checkpoint/\
2020-12-20_1926/params.pkl', 'wb') as f:
    pickle.dump(f, params)